## 1. Overview
This notebook demonstrates the use of a genetic algorithm to optimize quantum circuits. The algorithm searches for an optimal sequence of quantum gates to solve a specific quantum task, potentially defined by an oracle.

I am using these versions:
1. Qiskit version: 0.25.0
2. NumPy version: 1.23.5
3. Matplotlib version: 3.9.1

In [ ]:
from qiskit import *
import random
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import datetime



## 2. Define Global Constants and Circuit Parameters
This section defines constants and parameters used in quantum circuits, genetic algorithms, and quantum gate operations. It also sets the size of the quantum circuit, population size, mutation rate, and other parameters.


In [45]:
CircuitSize= 6 +1 # tha addetinal 1 is for the oracle output f(x)


number_of_total_gates=14 

LowerActionsLength = 2
UpperActionsLength = 20

POPULATION_SIZE = 6
survivors_to_next_generation= POPULATION_SIZE//10
GENERATIONS = 4
MUTATION_RATE = 0.01
EXTINCTION_RATE = 0.80

## 3. Controlled Gate Function
This function generates n-qubit controlled quantum gates and appends them to a circuit. The gate type is chosen based on the `target_gate` parameter, and it applies a controlled version of the gate using the control qubits.


In [46]:
def get_n_qubit_controlled_gate(circuit,target_gate,qubits,ctrl_state):
    cnx_circ = QuantumCircuit(1, name=target_gate)

    if target_gate=="x":
        cnx_circ.x(0)
    elif target_gate=="y":
        cnx_circ.y(0)
    elif target_gate=="z":
        cnx_circ.z(0)
    elif target_gate=="t":
        cnx_circ.t(0)
    elif target_gate=="tdg":
        cnx_circ.tdg(0)
    elif target_gate=="s":
        cnx_circ.s(0)
    elif target_gate=="sdg":
        cnx_circ.sdg(0)

    cnx = cnx_circ.to_gate().control(num_ctrl_qubits=qubits, ctrl_state=ctrl_state)
    circuit.append(cnx,[i for i in range(qubits+1)])

## 4. Convert Integer to Binary String
This helper function converts an integer to a binary string of a specified number of bits.


In [ ]:
def convert_to_binary_string(number, num_bits):
    binary = bin(number)[2:]
    binary = binary.zfill(num_bits)
    return binary

# Test the function
integer = 2
num_bits = 3
binary_string = convert_to_binary_string(integer, num_bits)
print(binary_string)



## 5. Oracle Construction
This function creates an oracle by adding quantum gates to the circuit. The oracle marks specific states based on a binary string (`oracleNumber`).


In [48]:


def oracle(oracleNumber,circuit,qubits):
    circuit.x(qubits)
    circuit.h(qubits)
    cnx_circ = QuantumCircuit(1, name="X")
    cnx_circ.x(0)
    cnx = cnx_circ.to_gate().control(num_ctrl_qubits=qubits, ctrl_state=oracleNumber)
    circuit.append(cnx,[i for i in range(qubits+1)])
    circuit.h(qubits)
    circuit.x(qubits)


## 6. Decoding Action into Quantum Gates
The `decodeAction_gates` and `decodeAction` functions translate an integer-based action into quantum gate operations that are applied to the circuit.


In [ ]:
def decodeAction(circuit, selector, oracleNumber, num_qubits):
    

    if selector == 0:#number_of_total_gates-1:
        oracle(oracleNumber, circuit, qubits=CircuitSize-1)
    if selector == 1:#21:
        get_n_qubit_controlled_gate(circuit=circuit,target_gate="z",qubits=CircuitSize-2,ctrl_state=str('1'*(CircuitSize-2)))
        # for gate_index in range(0,num_qubits//2):
        #     circuit.swap(gate_index, num_qubits-gate_index-1)
    # gate_index = selector % num_operations
    direction = 1 if (selector <20) else -1
    selector=selector%22
   
    for gate_index in range(0,num_qubits):
        if selector == 4:
            circuit.h(gate_index)
        elif selector == 5:
            circuit.x(gate_index)
        elif selector == 6:
            circuit.z(gate_index)
        elif selector == 7:
            circuit.t(gate_index) 
        elif selector == 8:
            circuit.y(gate_index)
        elif selector == 9:
            if gate_index != num_qubits-1:
                circuit.cx(gate_index, (gate_index+direction)%num_qubits)
        # elif selector == 6:
        #     circuit.cx(gate_index, (gate_index+direction)%num_qubits)
        elif selector == 10:
            if gate_index != num_qubits-1:
                circuit.cy(gate_index, (gate_index+direction)%num_qubits)
        # elif selector == 8:
        #     circuit.cy(gate_index, (gate_index+direction)%num_qubits)
        elif selector == 11:
            if gate_index != num_qubits-1:
                circuit.cz(gate_index, (gate_index+direction)%num_qubits)
        # elif selector == 10:
        #     circuit.cz(gate_index, (gate_index+direction)%num_qubits)
        # elif selector == 11:
        #     if gate_index != num_qubits-2:
        #         circuit.ccx(gate_index, (gate_index+direction)%num_qubits,(gate_index+2*direction)%num_qubits)
        # elif selector == 12:
        #     circuit.ccx(gate_index, (gate_index+direction)%num_qubits,(gate_index+2*direction)%num_qubits)
        elif selector == 12:
            circuit.sdg(gate_index)
        # elif selector == 14:
        #     if gate_index != num_qubits-2:
        #         circuit.ccz(gate_index, (gate_index+direction)%num_qubits,(gate_index+2*direction)%num_qubits)
        # elif selector == 15:
        #     circuit.ccz(gate_index, (gate_index+direction)%num_qubits,(gate_index+2*direction)%num_qubits)
        # elif selector == 16:
        #     circuit.s(gate_index)
        # elif selector == 17:
        #     if gate_index != num_qubits-1:
        #         circuit.swap(gate_index, (gate_index+direction)%num_qubits)
        # elif selector == 18:
        #     circuit.swap(gate_index, (gate_index+direction)%num_qubits)
        elif selector == 13:
            circuit.tdg(gate_index)
        # elif selector == 20:
        #     pass #do nothing
    
    
def get_all_possiable_gates():
    circuit = QuantumCircuit(CircuitSize , CircuitSize )
    for i in range(number_of_total_gates):
        decodeAction(circuit, i,None, num_qubits=CircuitSize-1)  # Example action
        circuit.barrier()
    return str(circuit)

print(get_all_possiable_gates())


## 6. Test for Identity Operation
This function tests whether a sequence of gate operations results in the identity matrix after multiple applications.


In [50]:
def test_identity_operation(num_qubits, max_trials=16):
    identity_matrix = np.eye(2**num_qubits)
    selectors_identity={}
    for select in range(number_of_total_gates):
        circuit = QuantumCircuit(CircuitSize)
        for i in range(max_trials):
            
            # Apply the decodeAction function multiple times
            decodeAction(circuit, select, '0'*(CircuitSize-1), num_qubits)  # Example usage, adjust as needed

            # Use the unitary simulator to get the unitary matrix of the circuit
            simulator = Aer.get_backend('unitary_simulator')
            job = execute(circuit, simulator)
            unitary = job.result().get_unitary(circuit)
            unitary=np.round(np.real(unitary),10)
            # Check if the unitary matrix is the identity matrix
            if np.allclose(unitary, identity_matrix):
                selectors_identity[select]=i + 1  # Return the count of applications needed for identity
                break
        
    return selectors_identity  # Return None if no identity found within max_trials



## 7. Filter Redundant Actions
This function removes redundant actions from a sequence based on identity matrix operations, simplifying the circuit.


In [51]:
def filter_the_action(action,identity_matrix_c):
    a_remove=[]
    pass_time=0
    for i in range(len(action)):
        if pass_time !=0:
                pass_time-=1
                continue
        if len(action)>(identity_matrix_c[action[i]]+i-1):
            temp=True
            for j in range(identity_matrix_c[action[i]]):
                temp = temp and (action[i] == action[i+j])
            if temp == True:
                pass_time=identity_matrix_c[action[i]]
                a_remove.append([i,action[i],identity_matrix_c[action[i]]])

    removed_items=0
    try:
        for r in a_remove:
            for k in range(r[2]):
                action.pop(r[0]-removed_items)
            removed_items+=r[2]
    except Exception as e:
        removed_items=[]

    return removed_items

In [ ]:
temp_action=[0,0,0,1,1,0,2,2,4,4,4,2,4,8,8,8,8,1,2,3,4,5,6,5,4,3,2,1,4,5,4]
identity_matrix_c = test_identity_operation(CircuitSize)
print(identity_matrix_c)
print(temp_action)
while filter_the_action(temp_action,identity_matrix_c) !=0:
    print(temp_action)



## 8. Generate Quantum Circuit
This function builds a quantum circuit by applying a sequence of actions. It decodes each action into a specific quantum gate operation and applies it to the quantum circuit.


In [53]:
def make_quantum_circuit(Actions,oracleNumber):
    
    circuit = QuantumCircuit(CircuitSize, CircuitSize-1)
    
    for action in Actions[:len(Actions)]:
        decodeAction(circuit,action,oracleNumber,num_qubits=CircuitSize-1) # the -1 is to not use the ancilla qubits

    return circuit

## 9. Calculate Circuit Output
This function calculates the output of a quantum circuit by running it on a statevector simulator. It returns the overlap between the quantum state's result and the expected output.


In [54]:

def calculateOutput(oracleNumber,trueOutput,actions):

         
    circuit = make_quantum_circuit(actions,oracleNumber)
    backend = BasicAer.get_backend('statevector_simulator') # the device to run on
    result = backend.run(transpile(circuit, backend)).result()
    psi  = result.get_statevector(circuit)

    return np.sum(np.dot(np.power(np.abs(psi),2),np.abs(trueOutput))),psi




## 10. Calculate the Corresponding Output Vector
This function creates an output vector based on the oracle's binary string. It sets the corresponding index in the output vector to `1`, indicating the expected output from the oracle.


In [ ]:
def calculateTheCorrespondingOutput(oracleNumber):
    # Calculate the total size of the system
    total_qubits = len(oracleNumber)  # Assumes oracleNumber length matches CircuitSize without ancilla qubits
    
    # Initialize the output vector with zeros
    output_vector_size = 2 ** (total_qubits + 1) # the +1 is for ancilla qubit of oracle
    output = [0] * output_vector_size
    
    # Convert oracleNumber from binary string to integer
    oracle_index = int(oracleNumber, 2)
    
    # Set the corresponding index in the output vector to 1
    output[oracle_index] = 1
    
    return output


calculateTheCorrespondingOutput("01")

In [56]:

def generate_random_circuit():
    # Implement the logic to generate a random quantum circuit
    # This function should return a QuantumCircuit object
    depth = random.randint(LowerActionsLength,UpperActionsLength)
    randomActions=[]
    for a in range(depth):
        randomActions.append(random.randint(0,number_of_total_gates-1))
    
    return randomActions



## 11. Genetic Algorithm Functions (Crossover, Mutation, Extinction)
These functions are responsible for the core components of the genetic algorithm, including creating offspring from parent circuits, mutating circuits, and applying extinction.


In [57]:
def crossover(parent1, parent2):
    # Implement the crossover logic to generate a new child circuit from two parent circuits
    # You can use standard genetic operators like single-point or uniform crossover
    LengthFactor = random.random()
    # childLength = round((len(parent1))
    child = []
    
    child.append(parent1[0:round(len(parent1)*LengthFactor)])

    child.append(parent2[round(len(parent2)*LengthFactor):])

    return sum(child,[])

In [58]:
def mutate(action):
    # Implement the mutation logic to introduce random modifications in the circuit
    # This function should modify the circuit in-place
    counter=0
    
    if (len(action)==0):
        action.insert(counter,random.randint(0,number_of_total_gates-1))
        action.insert(counter,random.randint(0,number_of_total_gates-1))
        action.insert(counter,random.randint(0,number_of_total_gates-1))

    for i in range(len(action)):
        
        if (random.random()<MUTATION_RATE):
            choise=random.random()
            if (choise<0.5):
                action[counter]=random.randint(0,number_of_total_gates-1)
            else:
                choise=random.random()
                if(choise<0.5):
                    action.insert(counter,random.randint(0,number_of_total_gates-1))
                else:
                    action.pop(counter)
                    counter-=1

        counter+=1



In [59]:

def extinction(actions, level=4):
    for action in actions:
        if random.random() <= EXTINCTION_RATE:
            if random.random() < 0.20:
                action= generate_random_circuit()
                break
            # If action is shorter than level, extend it
            while len(action) < level:
                action.insert(random.randint(0, max(0, len(action) - 1)), random.randint(0, number_of_total_gates - 1))

            choice = random.random()
            if choice < 0.5:
                # Insert new elements
                for i in range(level):
                    action.insert(random.randint(0, max(0, len(action) - 1)), random.randint(0, number_of_total_gates - 1))
            else:
                # Remove elements, ensuring action is not empty to avoid ValueError
                for i in range(min(level, len(action))):
                    if len(action) > 0:  # Check to ensure action is not empty
                        action.pop(random.randint(0, len(action) - 1))
            
    
    

In [ ]:
testActionsList =[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
testActionsList1 =[1,1,1,1,1,1,1]

print(crossover(testActionsList,testActionsList1))

In [ ]:
# for i in range(100):
extinction([testActionsList],4)
print(testActionsList)

## 12. Evaluate the Genetic Algorithm
This function evaluates a given set of actions by calculating the output of a quantum circuit for a set of oracle inputs. The reward is based on the correctness of the output compared to the expected result.


In [62]:



def evaluate_algorithm(actions,oracles,minimum=False):
    reward_array =[]
    reward=0
    for i in range(len(oracles)):
        # for q in range(CircuitSize-1):
            r,s = calculateOutput(oracleNumber=oracles[i],
                                  trueOutput=calculateTheCorrespondingOutput(oracles[i]),
                                  actions=actions)
            # print(r)
            reward+=r
            reward_array.append(r)

    if minimum==True:
        return min(reward_array)
    
    return round(reward,8)/(len(oracles))

## 13. Select Parents for the Next Generation
This function selects a subset of the population as parents based on their performance (fitness score). It ranks the population by the reward score and selects the top performers.


In [63]:

def select_parents(population,oracles,min=False,last_best_score=0):
    # Calculate the total fitness score of the population
    relative_fitness =[]
    memoization = {}
    print(len(oracles[0]))
    # Calculate the relative fitness of each circuit
    for actions in population:
        actions_key = tuple(actions)
        if actions_key not in memoization:
            oracle_index= random.randint(0,len(oracles)-1)
            action_score =evaluate_algorithm(actions=actions,oracles=[oracles[oracle_index]],minimum=min)
            if action_score>= 1 or action_score >last_best_score:
                test_oracles=[]
                for i in range(len(oracles[0])):
                    test_oracles.append(oracles[random.randint(0,len(oracles)-1)]) 
                action_score =evaluate_algorithm(actions=actions,oracles=test_oracles,minimum=True) 
            memoization[actions_key] = action_score
        
        relative_fitness.append(memoization[actions_key])
    
    # Sort the population based on the fitness scores in descending order
    
    combined = list(zip(population, relative_fitness))
    sorted_combined = sorted(combined, key=lambda x: x[1], reverse=True)
    sorted_population, sorted_scores = zip(*sorted_combined)
    
    # Select the top 50% of the population as parents
    num_parents = len(population) // 5
    parents = list(sorted_population)[:num_parents]
    return parents, list(sorted_scores)[:num_parents]


## 14. Generate Offspring
This function generates offspring for the next generation by combining parents using crossover and mutation. A certain number of the best parents are also retained to ensure high-performing circuits are preserved.


In [64]:
def generate_offspring(parents):
    offspring = []
    for _ in range(POPULATION_SIZE - survivors_to_next_generation - 1):
        parent1 = random.choice(parents)
        parent2 = random.choice(parents)
        child = crossover(parent1, parent2)
        mutate(child)
        offspring.append(child)
    return offspring



def generate_initial_population(size):
    return [generate_random_circuit() for _ in range(size)]

def print_to_file(test_name,generation, best_algorithm, best_score, avg_length, population_length, scores_avg,timestamp):
    with open(test_name+"_"+timestamp+'.txt', 'a') as file:
        print("Generation:", generation, file=file)
        print("Best algorithm (Generation", generation, "):", file=file)
        print(best_algorithm, file=file)
        print("Score:", str(best_score), file=file)
        print("Actions Avg lengths:", str(round(avg_length)),
              "   Population length:", str(population_length),
              "   Scores Avg:", str(scores_avg),
              "   MUTATION_RATE:", str(MUTATION_RATE), file=file)

def update_mutation_rate(last_best_score,best_score):
    global MUTATION_RATE
    if last_best_score == best_score:
        if MUTATION_RATE < 0.15:
            MUTATION_RATE += 0.0005
    else:
        MUTATION_RATE = 0.01

def get_best_algorithm_and_score(parents, scores):
    best_algorithm = parents[0]
    best_score = scores[0]
    for i in range(1, len(parents)):
        if abs(best_score - scores[i]) < 0.000001:
            if len(parents[i]) < len(best_algorithm):
                best_algorithm = parents[i]
    return best_algorithm, best_score

## 15. Start the Evolution Process
This function controls the evolutionary algorithm, running the circuit optimization process over a specified number of generations. It selects parents, generates offspring, and applies extinction when necessary. Results are printed and saved to a file.


In [65]:

def start_Evalotion(test_name,timestamp,oracles):

    population = generate_initial_population(POPULATION_SIZE)
    last_best_score = 0
    chance_for_imporvments=5

    list_of_all_final_algorithms=[]

   
    with open(test_name+"_"+timestamp+'.txt', 'w', encoding='utf-8') as file:
        print("Test name: " +test_name, file=file)
        print("Start Time and date: " +timestamp, file=file)
        print("input States: " + str(oracles), file=file)
        print("CircuitSize: " + str(CircuitSize), file=file)
        print("LowerActionsLength: " + str(LowerActionsLength), file=file)
        print("UpperActionsLength: " + str(UpperActionsLength), file=file)
        print("number_of_total_gates: " + str(number_of_total_gates), file=file)
        print("POPULATION_SIZE: " + str(POPULATION_SIZE), file=file)
        print("survivors_to_next_generation: " + str(survivors_to_next_generation), file=file)
        print("GENERATIONS: " + str(GENERATIONS), file=file)
        print("MUTATION_RATE: " + str(MUTATION_RATE), file=file)
        print("EXTINCTION_RATE: " + str(EXTINCTION_RATE), file=file)
        print("get_all_possiable_gates: \n" + str(get_all_possiable_gates()), file=file)
        
    try:
        for generation in range(1, GENERATIONS + 1):
            print("Generation:"+str( generation),end = "\t")

            for action in population:
                while filter_the_action(action,identity_matrix_c) !=0:
                    pass

            average_population_length = sum(len(gen) for gen in population) / POPULATION_SIZE

            parents, scores = select_parents(population=population, oracles=oracles)

            best_algorithm, best_score = get_best_algorithm_and_score(parents, scores)


            list_of_all_final_algorithms.append([best_algorithm, best_score])

            
            update_mutation_rate(last_best_score,best_score)

            scores_avg = round(sum(scores) / len(scores), 8)
            print_to_file(test_name,generation, best_algorithm, best_score, average_population_length, len(population), scores_avg,timestamp)

            offspring = generate_offspring(parents)

            if best_score == round(sum(scores) / len(scores), 8):
                extinction(offspring, level=max(CircuitSize,random.randint(CircuitSize,CircuitSize*10)))
                offspring[random.randint(0,len(offspring)-1)].pop()
                offspring.append(best_algorithm+best_algorithm)

            offspring += parents[:survivors_to_next_generation]
            offspring.append(best_algorithm)

            population = offspring

            last_best_score = best_score

            print("avg Population Length:"+str( average_population_length),end = "\t")
            print("Best Score:"+str(best_score),end = "\n")

            if best_score==1:
                chance_for_imporvments -=1
            if chance_for_imporvments==0:
                break
    except Exception as e:
        print("the traianing has been stopped >>>>> ")
        print(e)
        
    e = datetime.datetime.now()
    timestampEnd =  str(e).replace(":","_").split(".")[0]
    with open(test_name+"_"+timestamp+'.txt', 'a') as file:
        print("End Time and date: " +timestampEnd, file=file)
        print("Best Algoruthm: " + str(best_algorithm), file=file)
        print("Best Score: " + str(best_score), file=file)

    return list_of_all_final_algorithms

## 16. Save Circuit Image and Score Plots
These functions save images of quantum circuits and plots showing the evolution of scores over generations.


In [66]:
from qiskit.visualization import circuit_drawer
import os

def save_circuit_image_of_algorithm(algorithms,title,oracles,timestamp):
    # List to store the images
    

    old_score = 0
    old_algorithm=algorithms[0][0]

    # Create a directory to save the images
    os.makedirs("circuits_images_"+timestamp, exist_ok=True)

    for generation, algorithm in enumerate(algorithms[1:]):
        
        algorithm_score = algorithm[1]
        if algorithm_score > old_score:
            old_score=algorithm_score
        
            transpiled_circuit=make_quantum_circuit(Actions=old_algorithm,oracleNumber=oracles[0])
            score = evaluate_algorithm(actions=old_algorithm,oracles=oracles)
            # circuit_drawer(transpiled_circuit, filename='quantum_circuit.png', output='mpl')
            fig = circuit_drawer(transpiled_circuit, output='mpl')

            # Add a title to the plot
            fig.suptitle(title +" | Generation "+str(generation+1)+" | Score: "+ str(score), fontsize=16)
            fig.tight_layout()
            
            # Save the plot as a PNG file
            fig.savefig("circuits_images_"+timestamp+'\quantum_circuit_'+str(generation+1)+'.png')
        old_algorithm=algorithm[0]

    transpiled_circuit=make_quantum_circuit(Actions=algorithms[-1][0],oracleNumber=oracles[0])
    score = evaluate_algorithm(actions=old_algorithm,oracles=oracles)
    # circuit_drawer(transpiled_circuit, filename='quantum_circuit.png', output='mpl')
    fig = circuit_drawer(transpiled_circuit, output='mpl')

    # Add a title to the plot
    fig.suptitle(title +" | Generation "+str(len(algorithms))+" | Score: "+ str(score), fontsize=16)
    fig.tight_layout()

    # Save the plot as a PNG file
    fig.savefig("circuits_images_"+timestamp+'\quantum_circuit_'+str(generation-1)+'.png')



In [67]:
def save_scores_over_time_plot(list_of_all_final_algorithms, title,timestamp):
    generations = [i + 1 for i in range(len(list_of_all_final_algorithms))]
    scores = [pair[1] for pair in list_of_all_final_algorithms]

    # Plotting the scores over time
    plt.plot(generations, scores, marker='o')
    plt.title('Best Algorithm Scores Over Time: '+ title)
    plt.xlabel('Generation')
    plt.ylabel('Score')
    plt.grid(True)
    plt.savefig('best_algorithm_scores_over_time_'+timestamp+'.png')
    # plt.show()



## 17. Initialize States (Quantum Oracles)
This function initializes a list of possible oracle states for the quantum circuit. It generates binary strings representing all possible states for the given number of qubits.


In [68]:
def initialize_states(state_number,CircuitSize):
    oracles=[]
    for i in range(2**(CircuitSize-1)):
        oracleN = str(format(i, '0'+str(CircuitSize-1)+'b'))
        oracles.append(oracleN)
    print(oracles)
    return "Search Algorithm",oracles


## 18. Run the Experiment
This function is responsible for setting up and running an experiment. It uses the evolutionary algorithm to optimize the quantum circuits for the given oracles, saves circuit images, and generates plots for the results.


In [69]:
def run_an_experement(test_name,oracles):
    e = datetime.datetime.now()
    timestamp =  str(e).replace(":","_").split(".")[0]
    list_of_all_final_algorithms=[]
    
    list_of_all_final_algorithms = start_Evalotion(test_name,timestamp,oracles=oracles)
    min_algorithm=list_of_all_final_algorithms[-1][0]
    
        
    save_circuit_image_of_algorithm(algorithms=list_of_all_final_algorithms,title=test_name,oracles=oracles,timestamp=timestamp)
    # save_results_heatmap(list_of_best_algorithms=list_of_all_final_algorithms,timestamp=timestamp,input_states=input_states)
    save_scores_over_time_plot(list_of_all_final_algorithms,title=test_name,timestamp=timestamp)

## 19. Loop Through Different Circuit Sizes and Experiment Configurations
This section runs multiple experiments with different circuit sizes and configurations by creating folders and saving the results. It automates the process of testing different circuit sizes and running the genetic algorithm on each setup.


In [ ]:
for CircuitSize in range(3+1,5+2):  
    for experment_number in range(4):
        
        
        test_name,oracles = initialize_states(experment_number,CircuitSize)
        
        folder_name = test_name+'_'+str(CircuitSize-1)+'xx_folder'
        
        # Define the starting path
        start_path = r"C:\OraclesX"

        # Change the current working directory to the starting path
        os.chdir(start_path)
        print(f"Starting at: {os.getcwd()}")

        # Create a new folder
        os.makedirs(folder_name, exist_ok=True)
        print(f"Created folder: {folder_name}")

        # Navigate into the newly created folder
        os.chdir(folder_name)
        print(f"Current working directory: {os.getcwd()}")

        # Run your main function
        run_an_experement(test_name,oracles=oracles)



        # # Navigate back to the parent directory
        os.chdir('..')
        print(f"Back to the parent directory: {os.getcwd()}")



## 20. Extract and Plot Scores from Experiments
This block processes log files generated during the experiments, extracting scores from them and generating plots that show how the score evolved over generations. It also calculates average scores and top-performing scores across experiments.


In [ ]:
import os
import re
import random
import matplotlib.pyplot as plt



def extract_scores(file_path):
    scores = []
    generations = []
    with open(file_path, 'r') as file:
        content = file.read()
        matches = re.findall(r'Generation: (\d+).*?Score: ([\d\.]+)', content, re.DOTALL)
        if len(matches) >= GENERATIONS:  # Ensure the experiment has completed 100 generations
            for match in matches:
                generations.append(int(match[0]))
                scores.append(float(match[1]))
        else:
            print(f'File {file_path} does not have {GENERATIONS} generations.')
    return generations, scores

def plot_scores(folder_path, normalization_factor):
    log_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
    all_last_scores = []
    all_scores = []
    Roffset = 0
    
    for log_file in log_files:
        log_path = os.path.join(folder_path, log_file)
        generations, scores = extract_scores(log_path)
        if len(generations) == GENERATIONS:  # Only plot if there are 100 generations
            normalized_scores = [(score / normalization_factor) + Roffset for score in scores]
            all_last_scores.append((log_file, normalized_scores[-1]))
            all_scores.append((log_file, generations, scores, normalized_scores,Roffset))
        else:
            print(f'File {log_path} skipped because it does not have {GENERATIONS} generations.')
    print("all_last_scores:", all_last_scores)
    if all_last_scores:
        # Plot normalized scores
        plt.figure()
        for _, generations, _, normalized_scores,_ in all_scores:
            plt.plot(generations, normalized_scores)
        plt.xlabel('Generations')
        plt.ylabel('Normalized Scores')
        plt.title('Normalized Scores Over Generations')
        plt.grid(True)
        
        output_image_path = os.path.join(folder_path, 'normalized_scores_plot.png')
        plt.savefig(output_image_path)
        plt.close()

        # Plot non-normalized scores
        plt.figure()
        for _, generations, scores, _,_ in all_scores:
            plt.plot(generations, scores)
        plt.xlabel('Generations')
        plt.ylabel('Scores')
        plt.title('Scores Over Generations')
        plt.grid(True)
        output_image_path = os.path.join(folder_path, 'scores_plot.png')
        plt.savefig(output_image_path)
        plt.close()
    else:
        print(f'No valid experiments found in folder {folder_path}.')

    return all_last_scores, all_scores



def plot_combined_final_normalized_scores(all_scores):
    plt.figure()
    
    for qubits, scores in all_scores.items():
        final_scores = [score[3][-1] for score in scores]  # Access the non-normalized final scores
        final_scores.sort(reverse=True)
        plt.plot(range(1, len(final_scores) + 1), final_scores, label=f'{qubits} qubits')
    

    plt.xlabel('Experiments')
    plt.ylabel('Final Scores')
    plt.title('Final Normalized Scores for Each Experiment')
    plt.legend()
    plt.grid(True)
    plt.xticks(np.arange(1, len(final_scores) + 1, 1))
    # plt.yticks(np.arange(int(min(final_scores)), int(max(final_scores)) + 1, 1))
    output_image_path = os.path.join(r'C:\OraclesX', 'Final Normalized Scores for Each Experiment (Ordered)_plot.png')
    plt.savefig(output_image_path)
    plt.close()

def plot_combined_average_scores(all_scores):
    plt.figure()
    print(all_scores)
    for qubits, scores in all_scores.items():
        avg_scores_per_gen = []
        for gen in range(GENERATIONS):
            avg_score = sum(score[3][gen] for score in scores) / len(scores)
            avg_scores_per_gen.append(avg_score)
        plt.plot(range(1, GENERATIONS+1), avg_scores_per_gen, label=f'{qubits} qubits')
    
    plt.xlabel('Generations')
    plt.ylabel('Average Normalized Scores')
    plt.title('Average Normalized Scores Over Generations')
    plt.legend()
    plt.grid(True)
    
    output_image_path = os.path.join(r'C:\OraclesX', 'Average Normalized Scores Over Generations_plot.png')
    plt.savefig(output_image_path)
    plt.close()

#the ideal score for the first itteration of Grover Algorithm (used for normlization)
folders = {
    r'C:\OraclesX\Search Algorithm_3xx_folder': 0.78125,
    r'C:\OraclesX\Search Algorithm_4xx_folder': 0.47265625,
    r'C:\OraclesX\Search Algorithm_5xx_folder': 0.25830078125
    # r'C:\OraclesX\Search Algorithm_8xx_folder': 0.034790992734375
}

all_scores = {}

for folder, normalization_factor in folders.items():
    
    print(f'Processing folder: {folder}')
    last_scores, all_scores_for_folder = plot_scores(folder, normalization_factor)
    qubits = folder.split('_')[-2][0]  # Correctly extract the number of qubits from the folder name
    all_scores[int(qubits)] = all_scores_for_folder

plot_combined_final_normalized_scores(all_scores)
plot_combined_average_scores(all_scores)

print('Finished processing all folders.')
